## Structure Function Coupling & Cortical Gradients

- Author: A. Zarkali
- Date of last update: 14/05/2020
- Aim: Extract cortical-gradients

In [1]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import brainspace
from brainspace.gradient import GradientMaps
from brainspace.datasets import load_fsa5, load_conte69, load_parcellation
from brainspace.plotting import plot_hemispheres, plot_surf
from brainspace.utils.parcellation import map_to_labels
from brainspace.mesh import mesh_io as mio
import vtk
from pathlib import Path
import scipy.stats as stats
import statsmodels.api as sm
import nibabel
import nilearn
from nilearn import datasets, plotting, input_data, signal, image # datasets: for fetching atlas
import seaborn as sns
from brainspace.null_models import SpinPermutations
from brainspace.plotting import plot_hemispheres

In [4]:
### Participant Lists
# Load clinical information
clin = pd.read_excel(r"C:\Users\Angelika\Dropbox\PhD\PAPERS\Publications\Structure-Function\NatureComms\SupplementaryData1\Figure2\Data\ClinicalInformation.xlsx")
# Select controls only
controls = clin[clin.PD==0].ParticipantNumber
HighVis = clin[(clin.PD == 1) & (clin.PD_VisPerf == 0)].ParticipantNumber
LowVis = clin[(clin.PD == 1) & (clin.PD_VisPerf == 1)].ParticipantNumber

### Calculate template gradient

This is necessary so all resulting gradients are aligned

In [2]:
## Define cortical only indeces / columns
parcel = 401 ## parcellation cortical nodes + 1
cols = [] 
for i in range(1, parcel):
    cols.append(str(i))
index = []
for i in range(1,parcel):
    index.append(i)
num = [range(1,parcel)]
labelsNum = np.array(num)
labelsNum = labelsNum.reshape((parcel-1),)

In [ ]:
#Calculate group template gradient
## This is necessary to align all individuals to the same template
array = np.zeros((360, 360))
data_folder = Path(r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\04_StructureFunction\DATA\STRUCTURE")
for i in range(0, len(controls)):
    data = pd.read_csv(path, header=0, index_col=0)
    array = array + data
array = array / len(controls)
outfile = root / "Average_Connectome_400_controls.csv"
np.savetxt(outfile, array, delimiter=",")

In [ ]:
## Calculate template gradient
# Create a GradientMaps object for the template
Gt = GradientMaps(n_components=10, random_state=0, approach="dm", kernel="normalized_angle")
# Apply to template data
# Structural
#template_file = Path(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/DATA/STRUCTURE/Average_Connectome_400_Control.csv")
# Functional
template_file = Path(r"C:/Users/Angelika/Dropbox/PhD/EXPERIMENTS/04_StructureFunction/DATA/FUNCTION/Average_Connectome_400_bin.csv")
template_data = pd.read_csv(template_file, header=None)
template_data = pd.DataFrame(template_data, index=index)
template_data = template_data[index]
#template_data = template_data.replace(np.inf, 0) # for functional connectome need to replace the inf values to 0
Gt.fit(template_data.values, sparsity=0)

In [ ]:
## Plot eigenvalues of the average gradients for the template
fig, ax = plt.subplots(1, figsize=(10, 5))
ax.set_xlabel('Component Nb')
ax.set_ylabel('Eigenvalue')
colors = ["grey"]
sns.set_palette(sns.color_palette(colors))
sns.scatterplot(x=range(Gt.lambdas_.size), y=Gt.lambdas_)
plt.show()

In [ ]:
## Calculate and plot variance explained
var = []
for l in range(len(Gt.lambdas_)):
    v = Gt.lambdas_[l] / sum(Gt.lambdas_)
    var.append(v)

fig, ax = plt.subplots(1, figsize=(10, 5))
sns.scatterplot(x=range(10), y=var)
plt.show()

In [ ]:
## save eigenvalues and variance explained
np.savetxt("TemplateEigenvalues.txt", Gt.lambdas_)
np.savetxt("TemplateVariance.txt", var)

### Calculate individual gradients 

In [ ]:
# Create a GradientMaps object for the individuals
Gsub = GradientMaps(n_components=10, random_state=0, alignment="procrustes", approach="dm", kernel="normalized_angle")
# # Loop through control subjects and generate gradients

## Root directory
rootdir = Path(r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\04_StructureFunction\DATA")
## Loop through
grad1 = pd.DataFrame(data=np.zeros((400,0)))
grad2 = pd.DataFrame(data=np.zeros((400,0)))
for i in range(len(controls)):
    # for structural
#     filename = rootdir / "STRUCTURE" / All[i] / "shaefer_connectome_norm_400.csv"
#     conn_matrix = pd.read_csv(filename, index_col=0) # for structural
    # for functional
    filename = rootdir / "FUNCTION" / str(controls[i] + "func_bin.csv")
    conn_matrix = pd.read_csv(filename, index_col= 0) # for functional     
    # exclude the subcortical regions   
    conn_matrix = pd.DataFrame(conn_matrix, index=index)
    conn_matrix = conn_matrix[cols] # cols for structural and index for functional data
    conn_matrix = conn_matrix.replace([np.inf, np.nan], 0) # for functional data only need to replace inf values with 0 - diagonals
    ## convert to np.array
    conn_matrix = conn_matrix.values
    #Ask for 10 gradients (default)
    Gsub.fit([template_data.values, conn_matrix], sparsity=0) # no thresholding for structural connectomes as already sparse
    grad1[str(controls[i])] = (Gsub.aligned_[1])[:,0]
    grad2[str(controls[i])] = (Gsub.aligned_[1])[:,1]
#     print(All[i])
grad1 = grad1.transpose()
grad2 = grad2.transpose()
grad1.columns = [i for i in range(1,401)]
grad2.columns = [i for i in range(1,401)]
grad1.to_csv("FunctionalGradientOne.csv")
grad2.to_csv("FunctionalGradientTwo.csv")

In [ ]:
# Rank the gradients as per node position (from smallest to largest)
struct = pd.read_csv(r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\04_StructureFunction\DATA\GRADIENTS\StructuralGradientTwo.csv", index_col=0)
func = pd.read_csv(r"C:\Users\Angelika\Dropbox\PhD\EXPERIMENTS\04_StructureFunction\DATA\GRADIENTS\FunctionalGradientTwo.csv", index_col=0)
struct = struct.transpose()
func = func.transpose()

for i in range(len(controls)):
    sub = controls[i]
    func[sub] = func[controls[i]].rank(method="max")
    struct[sub] = struct[controls[i]].rank(method="max")
func.to_csv("FunctionalGradientTwo_Rank.csv")
struct.to_csv("FunctionalGradientTwo_Rank.csv")